In [1]:
from torch.utils import data
from torchvision import transforms
import torchvision

In [2]:

def load_data_fashion_mnist(batch_size, resize=None):
    
    trans = [transforms.ToTensor()]#转换为tensor
    if resize:
        trans.insert(0, transforms.Resize(resize))#resize操作可选
    trans = transforms.Compose(trans)#将转换操作聚合
    mnist_train = torchvision.datasets.FashionMNIST(root="../data",
                                                    train=True,
                                                    transform=trans,
                                                    download=True)
    mnist_test = torchvision.datasets.FashionMNIST(root="../data",
                                                   train=False,
                                                   transform=trans,
                                                   download=True)
    return (data.DataLoader(mnist_train, batch_size, shuffle=True,num_workers=4),data.DataLoader(mnist_test, batch_size, shuffle=False,num_workers=4))

In [3]:
train,test=load_data_fashion_mnist(4)

C:\Users\stepf\AppData\Local\conda\conda\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
img=train.__iter__().next()[0][0][0].numpy()
img.shape

(28, 28)

In [5]:
import matplotlib.pyplot as plt # plt 用于显示图片
import numpy as np
from PIL import Image
im = Image.fromarray(np.uint8(img*255))
im.show()


In [6]:
import torch.nn as nn
import torch.nn.functional as F


class block(nn.Module):
    def __init__(self):
        super(block,self).__init__()
        self.conv3=nn.Conv2d(16,16,3,padding=1)
        self.Conv4=nn.Conv2d(16,24,3,padding=1)
        self.Conv4=nn.Conv2d(24,16,3,padding=1)
    def forward(self,x):
        x=F.relu(self.fc2(x))
        return x

class Net(nn.Module):
    def __init__(self):
        #使用super()方法调用基类的构造器，即nn.Module.__init__(self)
        super(Net,self).__init__()
        # 1 input image channel ,6 output channels,3*3 square convolution kernel
        self.conv1=nn.Conv2d(1,6,3)#6,26,26
        # 6 input channl,16 output channels,3*3 square convolution kernel
        self.conv2=nn.Conv2d(6,16,3)#16,24,24
        self.bn1=nn.BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu=nn.ReLU()
        self.conv3=nn.Conv2d(16,24,3)#24,10,10
        self.conv4=nn.Conv2d(24,16,3)#16,8,8
        self.bn2=nn.BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        # an affine operation:y=Wx+b
        self.fc1=nn.Linear(16*4*4,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
        self.Softmax=nn.Softmax()
    def forward(self,x):
        # x是网络的输入，然后将x前向传播，最后得到输出
        # 下面两句定义了两个2x2的池化层
        x=self.conv1(x)
        x=F.max_pool2d(F.relu(self.conv2(x)),2)#16,12,12
        # if the size is square you can only specify a single number
        x=self.bn1(x)
        x=self.relu(x)
        x=self.conv3(x)
        x=F.max_pool2d(F.relu(self.conv4(x)),2)#16,4,4
        x=self.bn2(x)
        x=self.relu(x)
        x=x.view(-1,self.num_flat_features(x))
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return self.Softmax(x)
    def num_flat_features(self,x):
        size=x.size()[1:] # all dimensions except the batch dimension
        num_features=1
        for s in size:
            num_features*=s
        return num_features

In [7]:
import torch.optim as optim
model=Net()
optimizer=optim.Adam(model.parameters(),lr=0.001)
loss=nn.CrossEntropyLoss()



In [8]:
def accuracy(y_hat,y):#y_hat是预测的结果,y是已经编码的结果
    if len(y_hat.shape)>1 and y_hat.shape[1]>1:
        y_hat=y_hat.argmax(axis=1)
    cmp=y_hat.type(y.dtype)==y
    #print("y",y," y_hat",y_hat)
    return float(cmp.type(y.dtype).sum())

In [9]:
from tqdm import tqdm
def evaluate_accuracy(net,data_iter):
    if isinstance(net,nn.Module):
        net.eval()
    a=Accumulator(2)
    for X,y in data_iter:
        a.add(accuracy(net(X),y),len(y))#tensor.numel()返回tensor的元素个数
    return a[0]/a[1]

In [10]:
class Accumulator():
    def __init__(self,num):
        self.data=[0.0]*num
    def __getitem__(self,ind):
        return self.data[ind]
    def add(self,*argv):
        self.data=[a+float(b) for a,b in zip(self.data,argv)]
    def clear(self):
        self.data=[0.0]*len(self.data)

In [11]:
def train_epoch_ch3(net,train_iter,loss,updater):
    if isinstance(net,nn.Module):
        net.train()#将net的管理的参数梯度设为可改变
    matric=Accumulator(3)
    for X,y in tqdm(train_iter):#小批量迭代器
        y_hat=net(X)

        l=loss(y_hat,y)
        updater.zero_grad()
        l.backward()
        updater.step()
        #print(float(1)*l,accuracy(y_hat,y),len(y),y)
        matric.add(float(1)*l,accuracy(y_hat,y),len(y))
        
    return matric[0]/matric[2],matric[1]/matric[2]

In [12]:

def train_ch3(net, train_iter, test_iter, loss, num_epochs, updater):  #@save
    for epoch in range(num_epochs):
        train_metrics = train_epoch_ch3(net, train_iter, loss, updater)
        test_acc = evaluate_accuracy(net, test_iter)
        train_loss, train_acc = train_metrics
        print(train_loss, train_acc,test_acc)

In [13]:

train_ch3(model,train,test,loss,20,optimizer)

  0%|          | 0/15000 [00:00<?, ?it/s]C:\Users\stepf\AppData\Local\conda\conda\envs\pytorch\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
C:\Users\stepf\AppData\Local\Temp/ipykernel_24844/310693653.py:49: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.Softmax(x)
  0%|          | 0/15000 [00:00<?, ?it/s]

0.4258605686446031 0.7571666666666667 0.7856


  0%|          | 0/15000 [00:00<?, ?it/s]

0.4145475690762202 0.8020833333333334 0.8057


  0%|          | 0/15000 [00:00<?, ?it/s]

...

0.4116420346458753 0.8145333333333333 0.8384


  0%|          | 0/15000 [00:00<?, ?it/s]

0.4089118312438329 0.8254666666666667 0.8236


100%|██████████| 15000/15000 [03:23<00:00, 73.80it/s]


0.409984196271499 0.8211833333333334 0.8066
